In [8]:
from __future__ import print_function
import sys,os
import wave
import matplotlib.pyplot as plt
import scipy.signal as signal
from scipy.fftpack import fft
from scipy.io import wavfile # get the api
import pandas as pd
import seaborn as sns
import numpy as np
import sklearn
from tqdm import tqdm
%matplotlib inline
sns.set_style('whitegrid')
# import config 
import warnings
warnings.filterwarnings('ignore')


ng = u'\u014b'
n = 'n'
m = 'm'
ng_encoded = unichr(331).encode('utf-8')
nasal_phonemes = [ng, ng_encoded, m, n]


DATA_DIR = '/home/lvargas/shared/audio/guesswho/'
if sys.platform == 'darwin':
    DATA_DIR = '/Users/vargas/Desktop/mounts/kramerdata/guesswho/'
    
print('Data directory location: %s' % DATA_DIR)
    
def data_path(path):
    return os.path.join(DATA_DIR, path)

Data directory location: /Users/vargas/Desktop/mounts/kramerdata/guesswho/


In [2]:
df_acoustic = pd.read_csv(data_path('master_acoustic.csv'), sep=',',
       dtype = {
           'word' : np.str,
           'valid': np.str,
           'word_starttime': np.float64,
           'word_endtime': np.float64,
           'offset': np.float64,
           'duration': np.float64,
           'arpabet': np.str,
           'ipa': unicode,
           'phoneme_start': np.float64,
           'phoneme_end': np.float64,
           'filename': np.str,
           'sentence_number': np.float64,
           'global_path' : np.str,
           'sex' : np.str,
           'accent' :np.str,
           'id' : np.int
       })

df_praat = pd.read_csv(data_path('master_praat.csv'), sep=',',
       dtype = {
           'time': np.float64,
           'f1': np.float64,
           'f2': np.float64,
           'f3': np.float64,
           'sentence_number': np.float64,
           'global_path': np.str,
            'sex' : np.str,
           'accent' :np.str,
           'id' : np.int
       })

df_physical = pd.read_csv(data_path('master_physical.csv'), sep=',',na_values='-',
       dtype = {
           'id' : np.int,
           'sex' : np.str,
           'file' : np.str,
           'height' : np.float64,
           'face_height': np.float64,
           'pupil_height': np.float64,
           'interpupilary_dist': np.float64,
           'mouth_width': np.float64,
           'jaw_angle_width': np.float64,
           'eye_width': np.float64,
           'eye_height': np.float64,
           'mouth_height': np.float64,
           'lip_thickness': np.float64,
           'brow_height' : np.float64,
           'nose_height' : np.float64,
           'nose_width' : np.float64,
           'nose_depth' : np.float64
       })


df_voice = pd.read_csv(data_path('master_voice.csv'), sep=',',
       dtype = {
           'word' : np.str,
           'valid': np.str,
           'word_starttime': np.float64,
           'word_endtime': np.float64,
           'offset': np.float64,
           'duration': np.float64,
           'arpabet': np.str,
           'ipa': unicode,
           'phoneme_start': np.float64,
           'phoneme_end': np.float64,
           'filename': np.str,
           'sentence_number': np.float64,
           'global_path' : np.str,
           'sex' : np.str,
           'accent' :np.str,
           'id' : np.int,
           'time': np.float64,
           'f1': np.float64,
           'f2': np.float64,
           'f3': np.float64,
       })


In [9]:
df_voice['trim_ends'] = True
df_voice['step_difference'] = df_voice['phoneme_end'].shift(-1) - df_voice['phoneme_end']


for group, val in tqdm(df_voice.groupby(['id','sentence_number'])):
#     print(list(val.index.values))
    rows_removed = 2
    trim = 0
    row_indexes = list(val.index.values)
    for list_index,row_index in enumerate(row_indexes):
        # This trims the start of a phoneme
        if trim < rows_removed:
            df_voice.loc[row_index, 'trim_ends'] = False
            trim += 1

        # if this value is greater than 0, then the next row is the start
        # of another phoneme. This means that we need to start trimming the
        # end
        if df_voice.loc[row_index,'step_difference'] > 0:
            for idx in row_indexes[list_index - rows_removed +1 : list_index+1]:
#                 print(idx)
                df_voice.loc[idx, 'trim_ends'] = False
            trim = 0

100%|██████████| 2066/2066 [22:08<00:00,  2.28it/s]


In [ ]:
df.columns

In [ ]:
df = df_voice[(df_voice['id'] == 1) & (df_voice['sentence_number'] == 1)]

df['step_difference'] = df['phoneme_end'].shift(-1) - df['phoneme_end']
df['trim_ends'] = True
df = df[['time','sentence_number','word','ipa','phoneme_start','phoneme_end','word_starttime','word_endtime','step_difference','trim_ends']]


In [ ]:
def trim_phone_ends(df, rows_trimmed):
    df['trim_ends'] = True
    


    #         print(row_index, df.loc[row_index,'step_difference'])

#         print(list_index)
        
